In [51]:
# Install the required packages
!pip install scikit-learn matplotlib pandas openpyxl

# Set labels for each document

In [52]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
def get_similarity(query: str, document: list[str]) -> float:
    document_tf_idf = TfidfVectorizer().fit_transform(document)
    query_tf_idf = TfidfVectorizer().fit(document)
    query_tf_idf = query_tf_idf.transform([query])
    
    cosine_similarities = cosine_similarity(query_tf_idf, document_tf_idf).flatten()
    return max(cosine_similarities)

In [54]:
def encode_sentence(sentence: str, encoder_model) -> float:
    tfidf_array = encoder_model.transform([sentence]).toarray().flatten()
    return sum(tfidf_array) / len(tfidf_array)

In [55]:
# Load the datasets and use the keys to store the query
sheets = {}
queries = []
with pd.ExcelFile('data/loinc_dataset-v2_extended.xlsx') as xls:
    queries.extend(xls.sheet_names)
    for query in queries:
        sheets[query] = pd.read_excel(xls, query, skiprows=2, header=0, index_col=0)
        
sheets[queries[0]].head(10)

,long_common_name,component,system,property
loinc_num,,,,
1988-5,C reactive protein [Mass/volume] in Serum or P...,C reactive protein,Ser/Plas,MCnc
1959-6,Bicarbonate [Moles/volume] in Blood,Bicarbonate,Bld,SCnc
10331-7,Rh [Type] in Blood,Rh,Bld,Type
18998-5,Trimethoprim+Sulfamethoxazole [Susceptibility],Trimethoprim+Sulfamethoxazole,Isolate,Susc
1975-2,Bilirubin.total [Mass/volume] in Serum or Plasma,Bilirubin,Ser/Plas,MCnc
890-4,Blood group antibody screen [Presence] in Seru...,Blood group antibody screen,Ser/Plas,ACnc
20565-8,"Carbon dioxide, total [Moles/volume] in Blood",Carbon dioxide,Bld,SCnc
18906-8,Ciprofloxacin [Susceptibility],Ciprofloxacin,Isolate,Susc
2143-6,Cortisol [Mass/volume] in Serum or Plasma,Cortisol,Ser/Plas,MCnc


In [56]:
# Compute the similarity for each query
for query in queries:
    similarities = pd.Series([], dtype=float)
    for index in sheets[query].index:
        value = sheets[query].loc[index].values.flatten()
        similarities[index] = get_similarity(query, value)
    sheets[query]['similarity'] = similarities

In [57]:
# Save the results to a new file
with pd.ExcelWriter('data/loinc_dataset-v2_similarity.xlsx') as writer:
    for query in queries:
        sheets[query].to_excel(writer, sheet_name=query, startrow=2, startcol=0, index=True)

# Train a model to predict the similarity

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import root_mean_squared_error

In [59]:
seed = 42

In [60]:
models = {}
for query in queries:
    X = sheets[query].drop(columns=['similarity'])
    y = sheets[query]['similarity']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    X_encoded = pd.DataFrame()
    encoders = {}
    for column in X_train.columns:
        encoder = TfidfVectorizer()
        encoder.fit(X_train[column])
        X_encoded[column] = X_train[column].apply(lambda x: encode_sentence(x, encoder))
        encoders[column] = encoder
    
    # Train the model
    model = BayesianRidge()
    model.fit(X_encoded, y_train)
    
    X_test_encoded = pd.DataFrame()
    for column in X_test.columns:
        X_test_encoded[column] = X_test[column].apply(lambda x: encode_sentence(x, encoders[column]))
    
    # Predict the similarity
    y_pred = model.predict(X_test_encoded)
    
    # Evaluate the model
    error = root_mean_squared_error(y_test, y_pred)
    
    print(pd.DataFrame({'y_test': y_test, 'y_pred': y_pred}).head(10))
    
    models[query] = model
    print(f'Model for query = \"{query}\" obtains an error: {error:.2f}')
    
    print('-' * 75)
    

             y_test    y_pred
loinc_num                    
49926-9    0.470059  0.593749
49925-1    0.451421  0.590245
15076-3    0.619130  0.773227
14423-8    0.388614  0.521944
74774-1    0.582846  0.363484
23658-8    0.000000  0.445192
18928-2    0.000000  0.445192
59813-6    0.543812  0.557136
94231-8    0.468602  0.463473
27353-2    0.505900  0.571470
Model for query = "glucose in blood" obtains an error: 0.21
---------------------------------------------------------------------------
             y_test    y_pred
loinc_num                    
1003-3     0.478651  0.531625
14578-9    0.318388  0.479336
6768-6     0.492569  0.525876
1970-3     0.586489  0.501860
934-0      0.000000 -0.124642
883-9      0.508672  0.317459
26478-8    0.508672  0.293069
1988-5     0.525473  0.505067
8310-5     0.000000 -0.215619
1751-7     0.549584  0.506179
Model for query = "bilirubin in plasma" obtains an error: 0.16
---------------------------------------------------------------------------
     

In [61]:
X_new = pd.DataFrame({
    'loinc_num': ['12345-6', '12346-7', '12347-8'],
    'long_common_name': ['glucose in blood', 'bilirrubin in plasma', 'white blood cells count'],
    'component': ['glucose', 'bilirrubin', 'blood cells'],
    'system': ['blood', 'plasma', 'blood'],
    'property': ['mass', 'mass', 'count']
})
X_new.set_index('loinc_num', inplace=True)
X_new

,long_common_name,component,system,property
loinc_num,,,,
12345-6,glucose in blood,glucose,blood,mass
12346-7,bilirrubin in plasma,bilirrubin,plasma,mass
12347-8,white blood cells count,blood cells,blood,count


In [62]:
# Predict the similarity for the new documents
results = pd.DataFrame(X_new)
for query in queries:
    X_new_encoded = pd.DataFrame()
    for column in X_new.columns:
        X_new_encoded[column] = X_new[column].apply(lambda x: encode_sentence(x, encoders[column]))
    
    y_new_pred = models[query].predict(X_new_encoded)
    results[f'similarity_{query.replace(" ", "_")}'] = y_new_pred
    
results

,long_common_name,component,system,property,similarity_glucose_in_blood,similarity_bilirubin_in_plasma,similarity_White_blood_cells_count
loinc_num,,,,,,,
12345-6,glucose in blood,glucose,blood,mass,0.632106,-0.169923,0.200866
12346-7,bilirrubin in plasma,bilirrubin,plasma,mass,0.628064,-0.168258,0.054451
12347-8,white blood cells count,blood cells,blood,count,0.641887,-0.163000,0.262402
